## Introduction

- 「お弁当の需要予測」は実際にお昼に販売されたお弁当の売上数を予測する問題で、回帰問題として分類される
- まずはデータを読み込み、売上数と説明変数との関係を基礎分析で確認する

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
# スタイルの適用
sns.set(font="IPAexGothic", style="white")

In [4]:
train  = pd.read_csv("../00_Data/train.csv")
test   = pd.read_csv("../00_Data/test.csv")
sample = pd.read_csv("../00_Data/sample.csv", header=None)
print("Data Shape")
print("Train:", train.shape, "Test:", test.shape, "Sample:", sample.shape)

Data Shape
Train: (207, 12) Test: (40, 11) Sample: (40, 2)


In [5]:
train.index = pd.to_datetime(train["datetime"])
train.head()

,datetime,y,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature
datetime,,,,,,,,,,,,
2013-11-18,2013-11-18,90,月,0,厚切りイカフライ,NaN,NaN,NaN,NaN,快晴,--,19.8
2013-11-19,2013-11-19,101,火,1,手作りヒレカツ,NaN,NaN,NaN,NaN,快晴,--,17.0
2013-11-20,2013-11-20,118,水,0,白身魚唐揚げ野菜あん,NaN,NaN,NaN,NaN,快晴,--,15.5
2013-11-21,2013-11-21,120,木,1,若鶏ピリ辛焼,NaN,NaN,NaN,NaN,快晴,--,15.2
2013-11-22,2013-11-22,130,金,1,ビッグメンチカツ,NaN,NaN,NaN,NaN,快晴,--,16.1


In [6]:
train.describe()

,y,soldout,kcal,payday,temperature
count,207.000000,207.000000,166.000000,10.0,207.000000
mean,86.623188,0.449275,404.409639,1.0,19.252174
std,32.882448,0.498626,29.884641,0.0,8.611365
min,29.000000,0.000000,315.000000,1.0,1.200000
25%,57.000000,0.000000,386.000000,1.0,11.550000
50%,78.000000,0.000000,408.500000,1.0,19.800000
75%,113.000000,1.000000,426.000000,1.0,26.100000
max,171.000000,1.000000,462.000000,1.0,34.600000


In [8]:
# オブジェクト要素の基本統計量
train.describe(include="O")

,datetime,week,name,remarks,event,weather,precipitation
count,207,207,207,21,14,207,207
unique,207,5,156,6,2,7,8
top,2014-9-5,水,メンチカツ,お楽しみメニュー,ママの会,曇,--
freq,1,43,6,12,9,53,169


- 簡単な欠損値補間と月情報の特徴量の作成

In [ ]:
# 欠損値を0(非給料日)で補間
train["payday"] = train["payday"].fillna(0)
